In [1]:
import polars as pl
from minio import Minio
from datasets import load_dataset
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer

In [2]:
model_name = "facebook/nllb-200-distilled-600M"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to("cuda")

model

M2M100ForConditionalGeneration(
  (model): M2M100Model(
    (shared): M2M100ScaledWordEmbedding(256206, 1024, padding_idx=1)
    (encoder): M2M100Encoder(
      (embed_tokens): M2M100ScaledWordEmbedding(256206, 1024, padding_idx=1)
      (embed_positions): M2M100SinusoidalPositionalEmbedding()
      (layers): ModuleList(
        (0-11): 12 x M2M100EncoderLayer(
          (self_attn): M2M100Attention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
       

In [8]:
client = Minio(endpoint="storage.io", access_key="admin", secret_key="password", secure=False)

response = client.get_object("ecommerce", "raw/order_reviews.csv")

df = pl.read_csv(response.read())

df.head(200).write_csv("/tmp/order_reviews.head.csv")

dataset = load_dataset("csv", data_files="/tmp/order_reviews.head.csv")

dataset

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['review_id', 'order_id', 'review_score', 'review_comment_title', 'review_comment_message', 'review_creation_date', 'review_answer_timestamp'],
        num_rows: 200
    })
})

In [3]:
task = "translation_pt_to_en"

translator = pipeline(task, model=model, tokenizer=tokenizer, max_new_tokens=2048, device="cuda")

translator

In [4]:
translator("A compra foi realizada facilmente. A entrega foi efetuada muito antes do prazo dado. O produto já começou a ser usado e até o presente, sem problemas.")

[{'translation_text': 'en, a purchase was made easily, a delivery was made long before the deadline, the product was already in use, and up to now, without any problems.'}]

In [6]:
def translate(row):
    title = row["review_comment_title"]
    message = row["review_comment_message"]

    translated_title = title and translator(title)[0]["translation_text"]
    translated_message = message and translator(message)[0]["translation_text"]

    row["translated_review_comment_title"] = translated_title
    row["translated_review_comment_message"] = translated_message

    return row

In [9]:
translated_dataset = dataset["train"].map(translate)

translated_dataset

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (1024). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


Dataset({
    features: ['review_id', 'order_id', 'review_score', 'review_comment_title', 'review_comment_message', 'review_creation_date', 'review_answer_timestamp', 'translated_review_comment_title', 'translated_review_comment_message'],
    num_rows: 200
})

In [10]:
pl.Config.set_fmt_str_lengths(2000)

tdf = pl.DataFrame(translated_dataset.to_pandas())

tdf[["review_comment_message", "translated_review_comment_message"]].drop_nulls()

review_comment_message,translated_review_comment_message
str,str
"""Recebi bem antes do prazo estipulado.""","""ened before the deadline."""
"""Parabéns lojas lannister adorei comprar pela Internet seguro e prático Parabéns a todos feliz Páscoa""","""enkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenkenen"""
"""aparelho eficiente. no site a marca do aparelho esta impresso como 3desinfector e ao chegar esta com outro nome...atualizar com a marca correta uma vez que é o mesmo aparelho""","""enregistrement.no site a marque de l'appareil est imprimé comme 3desinfector et à la fin est avec un autre nom...atualizer comme une marque correcte étant donné qu'il est le même appareil."""
"""Mas um pouco ,travando...pelo valor ta Boa. ""","""en, travando, pelo valor de Boa. """
"""Vendedor confiável, produto ok e entrega antes do prazo.""","""en, product ok and delivery before deadline."""
…,…
"""Faltou 1 produto e os que recebi 1 veio quebrado""","""en 1 product e os que reciben 1 veio quebrado"""
"""chegou dentro do prazo e o produto é de excelente qualidade! Acabou as dores nas costas, estou dormindo muito bem!""","""en a la fecha y el producto es de excelente calidad! acabo como me duele las costas, estoy durmiendo muy bien!"""
"""aqui está descrevendo como entregue só que ate agora não recebi""","""en: here is describing how to deliver only what eats now don't get"""


### Not production ready